In [26]:
import os
import glob
from docx import Document
import pytesseract
from pdf2image import convert_from_path
import os
import subprocess
import re

def convert_docx_to_txt(docx_path):
    """
    Читает документ .docx и возвращает его содержимое в виде строки.
    """
    doc = Document(docx_path)
    full_text = [para.text for para in doc.paragraphs]
    text = "\n".join(full_text)
    return text

def convert_doc_to_txt(doc_path):
    try:
        result = subprocess.run(
            ["textutil", "-convert", "txt", "-stdout", doc_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        if result.returncode == 0:
            return result.stdout
        else:
            print("Ошибка:", result.stderr)
            return ""
    except Exception as e:
        print("Ошибка при запуске catdoc:", e)
        return ""

def ocr_pdf(pdf_path):
    """
    Takes a .pdf document and returns extracted text
    """
    os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'
    pages = convert_from_path(pdf_path, dpi=300, first_page=1, last_page=30)
    extracted_text = ''
    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page, lang='rus')
        extracted_text += text + ' \n'
    return extracted_text

def get_text_from_file(file_path):
    """
    Извлекает текст из файла с поддержкой форматов .txt, .docx и .pdf.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    
    if ext == ".txt":
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif ext == ".docx":
        return convert_docx_to_txt(file_path)
    elif ext == ".doc":
        return convert_doc_to_txt(file_path)
    elif ext == ".pdf":
        return ocr_pdf(file_path)
    else:
        print(f"Неподдерживаемый формат файла: {file_path}")
        return ""
    
pattern = re.compile(r'[^А-Яа-яЁё0-9\s\.,;:!?()\-\"]+')
def filter_russian_text(text):
    return pattern.sub('', text)

def normalize_whitespace(text):
    text = re.sub(r'\n\n+', '\n\n', text)
    text = re.sub(r'[ ]+', ' ', text)
    text = re.sub(r'\t+', '\t', text)
    return text.strip()


def process_documents(input_dir, output_dir):
    """
    Обрабатывает все файлы из input_dir с расширениями .docx, .pdf и .txt,
    извлекает из них текст и сохраняет в output_dir в виде файлов с нумерацией (1.txt, 2.txt, ...).
    """
    os.makedirs(output_dir, exist_ok=True)
    
    file_patterns = ["*.docx", "*.doc", "*.pdf", "*.txt"]
    files = []
    for pattern in file_patterns:
        files.extend(glob.glob(os.path.join(input_dir, pattern)))
    files.sort()

    i = 0
    for idx, file_path in enumerate(files, start=1):
        if i > 1e5:
            break
        print(f"Обработка файла: {file_path}")
        text = get_text_from_file(file_path)
        if text and text.strip():
            output_file = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(file_path))[0]}.txt")
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(normalize_whitespace(filter_russian_text(text)))
            # print(f"Сохранено: {output_file}")
        else:
            # print(f"Пустой текст или ошибка при обработке файла: {file_path}")
            pass
        i += 1

input_directory = "./rdata" 
output_directory = "./rproc_data"    

process_documents(input_directory, output_directory)


Обработка файла: ./rdata/1.docx
Обработка файла: ./rdata/10.docx
Обработка файла: ./rdata/100.docx
Обработка файла: ./rdata/1000.docx
Обработка файла: ./rdata/1001.docx
Обработка файла: ./rdata/1002.docx
Обработка файла: ./rdata/1003.docx
Обработка файла: ./rdata/1004.docx
Обработка файла: ./rdata/1005.docx
Обработка файла: ./rdata/1006.docx
Обработка файла: ./rdata/1007.docx
Обработка файла: ./rdata/1008.doc
Обработка файла: ./rdata/1009.doc
Обработка файла: ./rdata/101.docx
Обработка файла: ./rdata/1010.doc
Обработка файла: ./rdata/1011.docx
Обработка файла: ./rdata/1012.doc
Обработка файла: ./rdata/1013.docx
Обработка файла: ./rdata/1014.docx
Обработка файла: ./rdata/1015.docx
Обработка файла: ./rdata/1016.docx
Обработка файла: ./rdata/1017.docx
Обработка файла: ./rdata/1018.pdf
Обработка файла: ./rdata/1019.docx
Обработка файла: ./rdata/102.doc
Обработка файла: ./rdata/1020.docx
Обработка файла: ./rdata/1021.doc
Обработка файла: ./rdata/1022.docx
Обработка файла: ./rdata/1023.docx


In [14]:
%pip install textract --break-system-packages

Requested textract from https://files.pythonhosted.org/packages/6b/3e/ac16b6bf28edf78296aea7d0cb416b49ed30282ac8c711662541015ee6f3/textract-1.6.5-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Please use pip<24.1 if you need to use this version.
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      error in textract setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; .* suffix can only be used with `==` or `!=` operators
          extract-msg<=0.29.*
                     ~~~~~~~^
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12

In [11]:
import os

folder_path = './rdata'
allowed_extensions = ('.docx', '.doc', '.pdf')
i = 1

for filename in os.listdir(folder_path):
    old_file = os.path.join(folder_path, filename)
    if os.path.isfile(old_file):
        _, ext = os.path.splitext(old_file)
        if ext not in allowed_extensions:
            os.remove(old_file)
            continue
        new_name = f"{i}{ext}"
        new_file = os.path.join(folder_path, new_name)
        os.rename(old_file, new_file)
        # print(f"Переименован {old_file} -> {new_file}")
        i += 1


In [27]:
import os
import glob
import json
import time
import openai
from usecrets import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

prompt = (
    "Ты эксперт по суммаризации текста. Сформируй краткую, четкую и содержательную суммаризацию "
    "представленного текста, избегая шаблонных вводных фраз вроде 'Документ содержит' или 'В документе говорится'. "
    "Излагай все от третьего лица в нейтрально-официальном стиле. Ты должен написать не более 2-х предложений. Текст: "
)

def get_summary(text, max_retries=3):
    """
    Отправляет текст в ChatGPT API для получения суммаризации.
    При возникновении ошибок повторяет запрос с экспоненциальной задержкой.
    """
    for attempt in range(max_retries):
        try:
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model="o1-mini",
                messages=[{"role": "user", "content": prompt + text}],
                # temperature=0.3,
                max_completion_tokens=1000
            )
            summary = response.choices[0].message.content.strip()
            return summary
        except Exception as e:
            print(f"Ошибка при обработке текста: {e}. Попытка {attempt + 1} из {max_retries}.")
            time.sleep(2 ** attempt)
    return None

def process_files_and_save_summaries(input_dir, summaries_dir):
    """
    Обходит все .txt файлы в директории input_dir, проверяет наличие файла с суммаризацией
    (file_name_summ.txt) в директории summaries_dir. Если такого файла нет, запрашивает суммаризацию
    через ChatGPT API и сохраняет результат.
    """
    if not os.path.exists(summaries_dir):
        os.makedirs(summaries_dir)
        
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    
    for txt_file in txt_files:
        base_name = os.path.basename(txt_file)
        name, ext = os.path.splitext(base_name)
        summary_file = os.path.join(summaries_dir, f"{name}_summ.txt")
        
        if os.path.exists(summary_file):
            print(f"Файл {summary_file} уже существует. Пропускаем {txt_file}.")
            continue
        
        print(f"Обработка файла: {txt_file}")
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                text = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file}: {e}")
            continue

        if not text.strip():
            print(f"Пустой текст в файле {txt_file}. Пропускаем.")
            continue

        summary = get_summary(text)
        if summary:
            with open(summary_file, "w", encoding="utf-8") as f:
                f.write(summary)
            print(f"Суммаризация успешно сохранена для файла: {txt_file} -> {summary_file}")
        else:
            print(f"Не удалось получить суммаризацию для файла: {txt_file}")

def generate_training_dataset_from_summaries(input_dir, summaries_dir, output_file):
    """
    Обходит все .txt файлы в директории input_dir, для каждого находит соответствующий файл
    суммаризации (file_name_summ.txt) в summaries_dir и формирует пару {"text": original_text, "summary": summary}.
    Итоговый датасет сохраняется в output_file в формате JSON Lines.
    """
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    training_data = []
    
    for txt_file in txt_files:
        base_name = os.path.basename(txt_file)
        name, ext = os.path.splitext(base_name)
        summary_file = os.path.join(summaries_dir, f"{name}_summ.txt")
        
        if not os.path.exists(summary_file):
            print(f"Отсутствует суммаризация для файла: {txt_file}. Пропускаем.")
            continue
        
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                text = f.read()
            with open(summary_file, "r", encoding="utf-8") as f:
                summary = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file} или {summary_file}: {e}")
            continue
        
        if not text.strip() or not summary.strip():
            print(f"Пустой текст или суммаризация для файла: {txt_file}. Пропускаем.")
            continue
        
        training_data.append({
            "text": text,
            "summary": summary
        })
        print(f"Добавлена пара для файла: {txt_file}")
    
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in training_data:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + "\n")
    
    print(f"Обучающий датасет сохранён в файл: {output_file}")

# Пути к директориям и файлам
input_directory = "./rproc_data"         # исходная директория с .txt файлами
summaries_directory = "./rproc_data_summ"  # директория для сохранения суммаризованных файлов
output_jsonl = "train.jsonl"             # итоговый файл датасета

# Этап 1: Сохранение суммаризации для каждого файла (если ещё не обработан)
process_files_and_save_summaries(input_directory, summaries_directory)

# Этап 2: Генерация обучающего датасета из обработанных суммаризаций
generate_training_dataset_from_summaries(input_directory, summaries_directory, output_jsonl)


Обработка файла: ./rproc_data/1053.txt
Суммаризация успешно сохранена для файла: ./rproc_data/1053.txt -> ./rproc_data_summ/1053_summ.txt
Обработка файла: ./rproc_data/1735.txt
Суммаризация успешно сохранена для файла: ./rproc_data/1735.txt -> ./rproc_data_summ/1735_summ.txt
Обработка файла: ./rproc_data/1721.txt
Суммаризация успешно сохранена для файла: ./rproc_data/1721.txt -> ./rproc_data_summ/1721_summ.txt
Обработка файла: ./rproc_data/1047.txt
Суммаризация успешно сохранена для файла: ./rproc_data/1047.txt -> ./rproc_data_summ/1047_summ.txt
Обработка файла: ./rproc_data/1709.txt
Суммаризация успешно сохранена для файла: ./rproc_data/1709.txt -> ./rproc_data_summ/1709_summ.txt
Обработка файла: ./rproc_data/289.txt
Суммаризация успешно сохранена для файла: ./rproc_data/289.txt -> ./rproc_data_summ/289_summ.txt
Обработка файла: ./rproc_data/504.txt
Не удалось получить суммаризацию для файла: ./rproc_data/504.txt
Обработка файла: ./rproc_data/1090.txt
Суммаризация успешно сохранена дл

KeyboardInterrupt: 

In [36]:
import os
import glob
import json
import time
import openai
from usecrets import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
prompt = "Ты эксперт по суммаризации текста. Сформируй краткую, четкую и содержательную суммаризацию представленного текста, избегая шаблонных вводных фраз вроде 'Документ содержит' или 'В документе говорится'. Излагай все от третьего лица в нейтрально-официальном стиле. Ты должен написать не более 2-х предложений. Текст: "

def get_summary(text, max_retries=3):
    """
    Отправляет текст в ChatGPT API для получения суммаризации.
    При возникновении ошибок повторяет запрос с экспоненциальной задержкой.
    """
    for attempt in range(max_retries):
        try:
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model="o1-mini",
                messages=[
                    {"role": "user", "content": prompt + text}
                ],
                # temperature=0.3,
                max_completion_tokens=1000
            )
            summary = response.choices[0].message.content.strip()
            return summary
        except Exception as e:
            print(f"Ошибка при обработке текста: {e}. Попытка {attempt + 1} из {max_retries}.")
            time.sleep(2 ** attempt)
    return None

def generate_training_dataset(input_dir, output_file):
    """
    Обходит все .txt файлы в директории input_dir, считывает их содержимое,
    запрашивает суммаризацию через ChatGPT API и сохраняет итоговые пары {"text": ..., "summary": ...}
    в выходной файл output_file в формате JSON Lines.
    """
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    training_data = []
    
    for txt_file in txt_files:
        print(f"Обработка файла: {txt_file}")
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                text = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file}: {e}")
            continue

        if not text.strip():
            print(f"Пустой текст в файле {txt_file}. Пропускаем.")
            continue

        summary = get_summary(text)
        if summary:
            training_data.append({
                "text": text,
                "summary": summary
            })
            print(f"Суммаризация успешно получена для файла: {txt_file}")
        else:
            print(f"Не удалось получить суммаризацию для файла: {txt_file}")
    
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in training_data:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + "\n")
    
    print(f"Обучающий датасет сохранён в файл: {output_file}")


input_directory = "./rproc_data"
output_jsonl = "train.jsonl"
generate_training_dataset(input_directory, output_jsonl)


Обработка файла: ./proc_data/10.txt
Суммаризация успешно получена для файла: ./proc_data/10.txt
Обработка файла: ./proc_data/9.txt
Суммаризация успешно получена для файла: ./proc_data/9.txt
Обработка файла: ./proc_data/8.txt
Суммаризация успешно получена для файла: ./proc_data/8.txt
Обработка файла: ./proc_data/5.txt
Суммаризация успешно получена для файла: ./proc_data/5.txt
Обработка файла: ./proc_data/4.txt
Суммаризация успешно получена для файла: ./proc_data/4.txt
Обработка файла: ./proc_data/6.txt
Суммаризация успешно получена для файла: ./proc_data/6.txt
Обработка файла: ./proc_data/7.txt
Суммаризация успешно получена для файла: ./proc_data/7.txt
Обработка файла: ./proc_data/3.txt
Суммаризация успешно получена для файла: ./proc_data/3.txt
Обработка файла: ./proc_data/2.txt
Суммаризация успешно получена для файла: ./proc_data/2.txt
Обработка файла: ./proc_data/1.txt
Суммаризация успешно получена для файла: ./proc_data/1.txt
Обучающий датасет сохранён в файл: training_data.jsonl


In [19]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "Ты эксперт по суммаризации текста. Сформируй краткое и информативное резюме для представленного документа."
        },
        {"role": "user", "content": "тестовый текст"}
    ],
    temperature=0.3,
    max_tokens=256
)
summary = response.choices[0].message.content.strip()
summary

'Это тестовый текст, предназначенный для проверки работы системы суммаризации текста.'